<a href="https://colab.research.google.com/github/EmersonGodoi/Deteccao_Objetos_Video/blob/main/Deteccao_Objetos_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Etapa 1 - Importando as bibliotecas
import cv2
print(cv2.__version__)

import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile
print(cv2.__version__)

#Etapa 2 - Conectando com o Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

#Etapa 3 - Carregando os arquivos do modelo treinado
path =  '/content/gdrive/MyDrive/Detecção de Objetos com YOLO, Darknet, OpenCV e Python/modelo_YOLOv4.zip'
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

labels_path = os.path.sep.join(['/content/cfg', "coco.names"])
LABELS = open(labels_path).read().strip().split("\n")

weights_path = os.path.sep.join(['/content/', "yolov4.weights"])
config_path = os.path.sep.join(['/content/cfg', "yolov4.cfg"])

net = cv2.dnn.readNet(config_path, weights_path)

#Etapa 4 - Definindo mais configurações para a detecção
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

ln = net.getLayerNames()
print("Todas as camadas (layers):")
print(ln)
print("Total: "+ str(len(ln)))
print("Camadas de saída: ")
print(net.getUnconnectedOutLayers())
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

#Etapa 5 - Criando as funções para detecção e processamento do video
#Função para exibir imagens no Colab
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()
#Construindo o blob da imagem
def blob_imagem(net, imagem, mostrar_texto=True):
  inicio = time.time()
  blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)
  termino = time.time()
  if mostrar_texto:
    print("YOLO levou {:.2f} segundos".format(termino - inicio))
  return net, imagem, layerOutputs
#Realizando a detecção
def deteccoes(detection, _threshold, caixas, confiancas, IDclasses):
  scores = detection[5:]
  classeID = np.argmax(scores)
  confianca = scores[classeID]

  if confianca > _threshold:
      caixa = detection[0:4] * np.array([W, H, W, H])
      (centerX, centerY, width, height) = caixa.astype("int")

      x = int(centerX - (width / 2))
      y = int(centerY - (height / 2))

      caixas.append([x, y, int(width), int(height)])
      confiancas.append(float(confianca))
      IDclasses.append(classeID)

  return caixas, confiancas, IDclasses
#Mostrando o resultado da detecção no video
def funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=True):
  (x, y) = (caixas[i][0], caixas[i][1])
  (w, h) = (caixas[i][2], caixas[i][3])

  cor = [int(c) for c in COLORS[IDclasses[i]]]
  cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2)
  texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
  if mostrar_texto:
    print("> " + texto)
    print(x,y,w,h)
  cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

  return imagem,x,y,w,h

#Etapa 6 - Carregando o vídeo onde será feita a detecção
!cp /content/gdrive/MyDrive/Detecção\ de\ Objetos\ com\ YOLO,\ Darknet,\ OpenCV\ e\ Python/videos/video_pessoas01.mp4 ./
#Lendo o arquivo de vídeo com o OpenCV
arquivo_video = 'video_pessoas01.mp4'
cap = cv2.VideoCapture(arquivo_video)
conectado, video = cap.read()

conectado

video.shape

video_largura = video.shape[1]
video_altura = video.shape[0]
video_largura, video_altura

#Etapa 7 - Redimensionamento do tamanho do video
def redimensionar(largura, altura, largura_maxima = 600):
  if (largura > largura_maxima):
    proporcao = largura / altura
    video_largura = largura_maxima
    video_altura = int(video_largura / proporcao)
  else:
    video_largura = largura
    video_altura = altura

  return video_largura, video_altura

video_largura, video_altura = redimensionar(video.shape[1], video.shape[0])
print(video_largura,video_altura)

#Etapa 8 - Definindo as configurações do vídeo
nome_arquivo = 'resultado.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID') # MP4V

fps = 24 #video mais lento mudar para 20

saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

#Etapa 9 - Definindo as variáveis
threshold = 0.5
threshold_NMS = 0.3
fonte_pequena, fonte_media = 0.4, 0.6
fonte = cv2.FONT_HERSHEY_SIMPLEX

amostras_exibir = 20
amostra_atual = 0

#Etapa 10 - Processamento do vídeo e exibição do resultado
while (cv2.waitKey(1) < 0):
  conectado, frame = cap.read()
  if not conectado:
    break
  t = time.time()
  frame = cv2.resize(frame, (video_largura, video_altura))
  try:
    (H, W) = frame.shape[:2]
  except:
    print('Erro')
    continue

  imagem_cp = frame.copy()
  net, frame, layerOutputs = blob_imagem(net, frame)
  caixas = []
  confiancas = []
  IDclasses = []

  for output in layerOutputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():
      frame, x, y, w, h = funcoes_imagem(frame, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
      objeto = imagem_cp[y:y + h, x:x + w]

  cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t),
              (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)

  if amostra_atual <= amostras_exibir:
    cv2_imshow(frame)
    amostra_atual += 1

  saida_video.write(frame)

print('Terminou')
saida_video.release()
cv2.destroyAllWindows()

#Exportando Video
!du -h resultado.avi

!cp ./resultado.avi /content/gdrive/MyDrive/Detecção\ de\ Objetos\ com\ YOLO,\ Darknet,\ OpenCV\ e\ Python/videos/resultado3.avi





